<center>

## <center>Практическая работа по нейронным сетям

В работе использовался материал: [Виталия Радченко](https://github.com/Yorko/mlcourse_open/), Ю. В. Рубцова (Построение корпуса текстов для настройки тонового классификатора // Программные продукты и системы, 2015, №1(109), –С.72-78), [Никиты Учителева](https://habrahabr.ru/company/dca/blog/274027/), [Exploring LSTMs](http://blog.echen.me/2017/05/30/exploring-lstms/), [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)


#  Нейронные сети в решении задач сентимент-анализа


## Постановка задачи
Для многих целей важно знать, что о пишут в интернете по тому или иному вопросу. В работе рассмотрена задача классификация отзывов в интернете на позитивные и негативные, на языке машинного обучения это будет означать решение задачи бинарной классификации.

## Данные

В качестве источника текстов была выбрана платформа микроблогинга Twitter. Каждый текст в корпусе имеет следующие атрибуты:

    – дата публикации;
    – имя автора;
    – текст твита;
    – класс, к которому принадлежит текст (положительный, отрицательный, нейтральный);
    – количество добавлений сообщения в избранное;
    – количество ретвитов (количество копирований этого сообщения другими пользователями);
    – количество друзей пользователя;
    – количество пользователей, у которых данный юзер в друзьях (количество фоловеров);
    – количество листов, в которых состоит пользователь.

 

В результате используется тренировочный корпус, состоящий из 114,911 положительных, 111,923 отрицательных записей.


## Подходы к решению

Поскольку эта классическаязадача бинарной классификации, ее можно решать массой разных способов:
- линейные модели ( логистическая регрессия, SVM, Naive Bayes и др.);
- деревья, ансамбли, бустинг (дерево решений, случайный лес, Xgboost и др.);
- Библиотека Facebook [FastText](https://github.com/facebookresearch/fastText);
- нейронные сети на словах и символах (рекуррентные, LSTM, GRU, CNN и др.).

Мы рассмотрим только решение, основанное на нейронных сетях. 
Итак, загрузим наши данные.

In [18]:
import pandas as pd
import numpy as np
from pprint import pprint

positive = pd.read_csv('positive.csv', sep=';', header=None)
negative = pd.read_csv('negative.csv', sep=';', header=None)

Посмотрим, как выгледят наши данные:

In [2]:
positive.tail()

,0,1,2,3,4,5,6,7,8,9,10,11
114906,411368729235054592,1386912922,diminlisenok,"Спала в родительском доме, на своей кровати......",1,0,0,0,1497,56,34,2
114907,411368729424187392,1386912922,qilepocagotu,RT @jebesilofyt: Эх... Мы немного решили сокра...,1,0,1,0,692,225,210,0
114908,411368796537257984,1386912938,DennyChooo,"Что происходит со мной, когда в эфире #proacti...",1,0,0,0,4905,448,193,13
114909,411368797447417856,1386912938,bedowabymir,"""Любимая,я подарю тебе эту звезду..."" Имя како...",1,0,0,0,989,254,251,0
114910,411368857035898880,1386912953,Prituljak_Sibir,@Ma_che_rie посмотри #непытайтесьпокинутьомск ...,1,0,0,0,1005,221,178,6


Нам нужна только колонка 3 в данных, которая содержит текст, а также метка с тональностью (1 - позитивно, 0 - негативно):


In [3]:
data = pd.DataFrame(columns =['text', 'sentiment'])
data.loc[:, 'text'] = pd.concat([positive[3], negative[3]], axis=0, ignore_index=1)
data.loc[0:len(positive)-1, 'sentiment'] = 1
data.loc[len(positive):, 'sentiment'] = 0

In [4]:
print(data.head())
print(data.tail())

                                                text sentiment
0  @first_timee хоть я и школота, но поверь, у на...         1
1  Да, все-таки он немного похож на него. Но мой ...         1
2  RT @KatiaCheh: Ну ты идиотка) я испугалась за ...         1
3  RT @digger2912: "Кто то в углу сидит и погибае...         1
4  @irina_dyshkant Вот что значит страшилка :D\nН...         1
                                                     text sentiment
226829  Но не каждый хочет что то исправлять:( http://...         0
226830  скучаю так :-( только @taaannyaaa вправляет мо...         0
226831          Вот и в школу, в говно это идти уже надо(         0
226832  RT @_Them__: @LisaBeroud Тауриэль, не грусти :...         0
226833  Такси везет меня на работу. Раздумываю приплат...         0


Далее надо немного обработать корпус и разбить выборку на тренировочную и тестовую.

In [5]:
import re

from sklearn.model_selection import train_test_split

# инициализируем параметры 
VALIDATION_SPLIT = 0.1
RANDOM_SEED = 42

# приводим слова к нижнему регистру и убираем лишние символы
data['text'] = data['text'].apply(lambda r: r.lower())
data['text'] = data['text'].apply(lambda r: re.sub(r'[^а-я]+', ' ', r))
 
# разделение выборки на тренировочную и тестовую
data_train, data_test, label_train, label_test = \
    train_test_split(data['text'], data['sentiment'],
                     test_size=VALIDATION_SPLIT, random_state=RANDOM_SEED)

In [42]:
print(data.head())

                                                text sentiment
0   хоть я и школота но поверь у нас то же самое ...         1
1  да все таки он немного похож на него но мой ма...         1
2                ну ты идиотка я испугалась за тебя          1
3   кто то в углу сидит и погибает от голода а мы...         1
4   вот что значит страшилка но блин посмотрев вс...         1


## Предобработка данных

В "сыром" виде нельзя подавать данные в модель, их нужно правильно подготовить. 
Давайте заменим одинаковые слова на одинаковые числовые коды, редкие слова учитывать не будем, а также обрежем длину каждого предложения до 100 слов.


In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# инициализируем параметры словаря и эмбеддингов
MAX_NB_WORDS = 100000
MAX_SEQUENCE_LENGTH = 100

print("Предложение до предобработки:\n", data_train[42])


# с помощью Tokenizer создаем словарь 
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(data['text'])

# заменяем слова на их индексы в нашем словаре
X_train = tokenizer.texts_to_sequences(data_train)
X_test = tokenizer.texts_to_sequences(data_test)

print("Предложение после замены слов на индексы:\n", X_train[42])

# обрезаем каждое предложение приводим к нужной длинне
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

print("Предложение после приведения к единой длинне:\n", X_train[42])

Using TensorFlow backend.


Предложение до предобработки:
 как же мне нравятся мужчины в строгих костюмах млею 
Предложение после замены слов на индексы:
 [152, 19, 24, 61]
Предложение после приведения к единой длинне:
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0 152  19  24  61]


### Задание 1

Объект `Tokenizer` хранит в себе всю информацию про наш словарь. Нужно найти индекс слова "сегодня" и сколько раз оно встречалось в нашей выборке.

In [48]:
# !pip3 install re #Запуск консольных команд
tokenizer?? #Вывод исходников по чему угодно

  Could not find a version that satisfies the requirement re (from versions: )
No matching distribution found for re


In [55]:
#print(tokenizer.__dict__.keys())
tokenizer.

In [56]:
# ЗАМЕНИТЕ ?? НА ПРАВИЛЬНЫЙ ОТВЕТ
s_index = tokenizer.word_index['сегодня']
s_count = tokenizer.word_counts['сегодня']
print("Индекс слова 'сегодня' – {}.".format(s_index))
print("Слово 'сегодня' встречалось {} раз.".format(s_count))
s_index = tokenizer.word_index['вчера']
s_count = tokenizer.word_counts['вчера']
print("Индекс слова 'вчера' – {}.".format(s_index))
print("Слово 'вчера' встречалось {} раз.".format(s_count))

Индекс слова 'сегодня' – 27.
Слово 'сегодня' встречалось 8800 раз.
Индекс слова 'вчера' – 131.
Слово 'вчера' встречалось 1897 раз.


## Рекуррентные нейронные сети (Recurrent neural networks, RNN)

Рекуррентные нейронные сети помогают уловить/понять закономерность, которая зависит от времени или порядка. Например, когда мы пытаемся классифицировать какой-то эпизод из фильма, то нам важно знать что было пару эпизодов ранее, или чтобы понять смысл определенного слова, нам нужно знать контекст, который был до него.

Простая рекуррентная нейронная сеть имеет следующее математическое представление:<br><br>
$$\large h_t = \phi(Wx_t + Uh_{t-1})$$<br>
$$\large y = Vh_t$$

Илюстрация к формуле:
<img src="http://i.imgur.com/ifQrKRR.png" alt="rnn" style="width: 700px;"/>

In [38]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Embedding
from keras.layers import SimpleRNN
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

max_features = 100000
maxlen = 100 
                            
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SimpleRNN(100))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 128)          12800000  
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 100)               22900     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 12,823,001
Trainable params: 12,823,001
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.fit(X_train, label_train, validation_data=[X_test, label_test], 
          batch_size=32, epochs=1)

Train on 204150 samples, validate on 22684 samples
Epoch 1/1
204150/204150 [==============================] - 2042s - loss: 0.5538 - acc: 0.7145 - val_loss: 0.5674 - val_acc: 0.7241


Если натренировать данную модель, то точность на тренировочной выборке получится примерно 71.03%, а на валидации – примерно 74.59%.

## Long short-term memory (LSTM)

LSTM имеет ряд приемуществ над простой рекуррентной нейронной сетью. LSTM умеет хранить нужную информацию про определенный объект и не обращать внимание на неактуальную информацию. Например, сцена в книге без упоминания главного героя не будет менять информацию про него и, наоборот, при упоминании она будет фокусироваться. Рассмотрим на примере тренировки сети на тексте книги.

- **Добавление механизма забывания.** Если эпизод книги заканчивается, то модель должна забыть текущее местоположение, время суток и сбросить любую информацию о конкретной сцене. Однако если персонаж книги умирает в сцене, сеть должна должна продолжать помнить, что он больше не жив. Таким образом, мы хотим, чтобы модель изучила отдельный механизм забывания/запоминания: когда появляются новые входные данные, она должна знать, какие факты сохранить или выбросить.

- **Добавление механизма сохранения.** Когда модель увидит новую сцену, ей необходимо решить, стоит ли использовать и сохранять какую-либо информацию о ней. 

- Поэтому когда приходит новый вход, модель сначала забывает долгосрочную информацию, которая, как она решает, больше не нужна. Затем она узнает, какую часть новых данных стоит использовать, и сохраняет ее в своей долгострочной памяти.

- **Фокусировка с долгосрочной памяти в рабочую память.** Наконец, модель должна узнать, какие части ее долговременной памяти сейчас полезны. Например, возраст героя может быть полезной информацией для сохранения в долгосрочной перспективе (дети с большей вероятностью будут ползать, взрослые скорее всего будут работать), но, вероятно, не имеет значения, если он не находится в текущей сцене. Таким образом, вместо того чтобы использовать полную долгосрочную память все время, она узнает, на каких частях стоит сосредоточиться.

То есть преимущество LSTM над RNN в том, что RNN может только перезаписывать память, а LSTM более гибкая в этом плане и может хранить долгосрочную информацию, фокусируясь на нужных ее частях.

Давайте рассмотрим игрушечный пример для закрепления понимания. 

Что "думает" полносвязная нейронная сеть:
<img src="http://i.imgur.com/cOGzJxk.png" alt="pokemon_nn" style="heigh: 100px;"/>

Что "думает" простая рекуррентная сеть:
<img src="http://i.imgur.com/PnWiSCf.png" alt="pokemon_rnn" style="heigh: 100px;"/>

Как видно из рисунка, рекуррентная сеть помнит, что случилось пару секунд назад, и может примерно понять, что стало причиной появления воды в следующем кадре.

Что "думает" LSTM:
<img src="http://i.imgur.com/EGZIUuc.png" alt="pokemon_lstm" style="heigh: 100px;"/>

LSTM вспоминает, что было в предыдущем эпизоде, а также подтягивает долгосрочную память и фокусируется только на нужной для конкретного эпизода информации.



Итак, приступим к тренировке LSTM сети:

In [40]:
from keras.layers import LSTM, Dropout
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))

# Изменим архитектуру сети, добавив еще один слой.
# Кроме того, будем отключать небольшую долю случайных нейронов сети для того,
# чтобы она лучше обучалась: эта методика называется Dropuot

model.add(Dropout(0.2))

# обратите внимание: для того, чтобы результат парвого слоя LSTM 
# использовать в следующем слое LSTM, необходимо добавить инструкцию
# return_sequences=True

model.add(LSTM(100, dropout=0.1, recurrent_dropout=0.1, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(32, dropout=0.1, recurrent_dropout=0.1))
model.add(Dropout(0.2))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()
model.fit(X_train, label_train, validation_data=[X_test, label_test],  
          batch_size=32, epochs=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 128)          12800000  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 100)          91600     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                17024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
__________

При замене простой RNN на более сложную LSTM точность на тренировочной выборке возросла до 73.03%, а на валидации – до 75.70%.


### Задание 2

Попробуйте, варьируя параметры нейронной сети, поднять точность модели. Какой точности вы смогли добиться на валидационной выборке?  


Другие варианты улучшения точности для LSTM модели в задаче сентимент анализа:
- использование эмбеддингов;
- увеличение размерности выхода ячейки LSTM;
- на больших данных работает увеличение к-ва слоев;
- переход от маленького батча к большому в процессе обучения;
- подбор гиперпараметров для дропаута, регуляризации и оптимизатора.


Еще лучше с задачей сентимент-анализа справляются сверточные сети.
У сверточных нейронных сетей есть несколько преимуществ перед LSTM:
- не нужно хранить тысячи слов, а только небольшое к-во символов;
- опечатки практически не влияют на точность модели ("the bst film" будет классифицирован как очень хороший, а LSTM просто проигнорирует данное слово).
Однакое в данной работе мы ограничимся знакомством только с рекурентными сетями. 

Практические наблюдения:
- если мало данных и отзывы короткие, то лучше использовать линейные методы (логистическая регрессия / SVM / etc);
- если мало данных, но отзывы длинные – хорошо подойдет однослойная LSTM;
- много данных – стоит пробовать разные архитектуры сети LSTM, CNN, а так же их модификации.



Теперь проверим нашу модель в действии!
Для этого нужно повторить все шаги подготовки, которые переводят текст в последовательность чисел. 

In [41]:
your_text = 'не очень хорошо'
test_text = tokenizer.texts_to_sequences([your_text])
test_train = pad_sequences(test_text, maxlen=MAX_SEQUENCE_LENGTH)
model.predict(test_train)[0][0]

0.48138702

Нам может показаться этот пример простым, но в нем есть большая сложность для нейронной сети: она должна научиться праввильно понимать отрицание. Как мы можем видеть, приведенный пример она понимает корректно.

### Задание 3

- Предложите натренированной сети несколько выражений из словаря [Эллочки-людоедки](https://ru.wikipedia.org/wiki/%D0%AD%D0%BB%D0%BB%D0%BE%D1%87%D0%BA%D0%B0-%D0%BB%D1%8E%D0%B4%D0%BE%D0%B5%D0%B4%D0%BA%D0%B0). Учтите, что сеть тренировалась на словах в нижнем регистре и состоящих только из букв, без дефисов и других дополнительных символов. Как Вы считаете, сможет ли сеть понять Эллочку? 
- Приведите пример выражений, которые сеть точно понимает неправильно. Объясните, почему сеть не смогла понять правильно найденные примеры?
- Попробуйте подобрать максимально положительное и отрицательное выражение с точки зрения нейронной сети. 
